<a href="https://colab.research.google.com/github/Ishan1440/Survey-Response-Quality-Analysis/blob/main/superficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load and preprocess data
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


import pandas as pd
import ast
import re
df = pd.read_csv('squad_superficiality_labeled.csv')

# Rename target column to 'labels'
df = df.rename(columns={'superficiality_score': 'labels'})
df['labels'] = df['labels'].astype('float32')

# Extract answer text from JSON
def extract_answer_text(example):
    answers = ast.literal_eval(example['answers'])
    example['answer_text'] = answers['text'][0] if answers['text'] else ""
    return example

df = df.apply(extract_answer_text, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples['question'],
        examples['answer_text'],
        padding="max_length",
        truncation=True,
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split dataset
split_datasets = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']

# Load model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=1  # Regression
)

# Configure TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sanyamsankhala468 (sanyamsankhala468-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.018500,0.029516
2,0.002900,0.016479
3,0.000900,0.009352


TrainOutput(global_step=2628, training_loss=0.022137351685644468, metrics={'train_runtime': 2215.8372, 'train_samples_per_second': 9.488, 'train_steps_per_second': 1.186, 'total_flos': 5531597161463808.0, 'train_loss': 0.022137351685644468, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./superficiality_model")
tokenizer.save_pretrained("./superficiality_model")

('./superficiality_model/tokenizer_config.json',
 './superficiality_model/special_tokens_map.json',
 './superficiality_model/vocab.txt',
 './superficiality_model/added_tokens.json')

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load model and tokenizer
model_path = "/content/superficiality_model"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Load base tokenizer
model = BertForSequenceClassification.from_pretrained(model_path).to("cuda")

def predict_superficiality(question: str, answer: str) -> float:
    # Combine question and answer
    text_pair = [question, answer]

    # Tokenize with proper parameters
    inputs = tokenizer(
        text_pair[0],
        text_pair[1],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    ).to("cuda")  # Ensure inputs match model device

    # Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert raw logits to 0-1 score
    score = torch.sigmoid(outputs.logits).item()
    return round(score, 4)

# Validation examples
test_cases = [
    ("Q16B. What is the most important thing you DISLIKE about the shown concept}?    This can include general concerns, annoyances, or any aspects of the product that need fixed for this to be more appealing to you...     Please type a detailed response in the space below.",
     "Bitterness",
     "Expected low score (deep)"),

    ("Did you like the movie?",
     "It was okay.",
     "Expected high score (superficial)"),

    ("How does photosynthesis work?",
     "Plants use sunlight to make food.",
     "Expected moderate score")
]

# Run tests
for question, answer, description in test_cases:
    score = predict_superficiality(question, answer)
    print(f"{description}: {score}")

Expected low score (deep): 0.6517
Expected high score (superficial): 0.7298
Expected moderate score: 0.4887
